In [ ]:
# -*- coding: utf-8 -*-
import re
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from google.colab import files
import unicodedata as ud

# === Config knobs ===
AXIS_LINE            = 5     # axis/tick thickness
MARKER_SIZE          = 120     # scatter marker area (points^2) — plot markers
LEGEND_MARKERSCALE   = 2.2    # legend marker size only (relative to default)
FIG_LABEL_TEXT       = "(a)"  # figure label
FIG_LABEL_COLOR      = "#000080"  # vivid, distinct
FIG_LABEL_SIZE       = 60

# === 1) Upload workbook (Colab) ===
print("📁 UPLOAD your workbook (e.g., Hardness_SDSS_3% Ni.xlsx) NOW:")
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded.")
xlsx_candidates = sorted(Path('.').glob('*.xlsx'),
                         key=lambda p: p.stat().st_mtime, reverse=True)
filename = str(xlsx_candidates[0])
print(f"📦 Using file on disk: {filename}")

# === 2) Header utilities ===
def _norm(s: str) -> str:
    s = str(s)
    s = s.strip().lower().replace(",", ".")
    s = re.sub(r"\s+", " ", s)
    s = s.replace("–", "-").replace("—", "-")
    s = "".join(c for c in ud.normalize("NFD", s) if ud.category(c) != "Mn")
    return s

def resolve_col(columns, candidates):
    norm_map = {_norm(c): c for c in columns}
    # exact normalized match
    for cand in candidates:
        key = _norm(cand)
        if key in norm_map:
            return norm_map[key]
    # relaxed: ignore non-word characters
    for cand in candidates:
        key = re.sub(r"[^\w]", "", _norm(cand))
        for k, orig in norm_map.items():
            if key in re.sub(r"[^\w]", "", k):
                return orig
    raise KeyError(f"Could not find any of {candidates} in: {list(columns)}")

DIST_CANDS = [
    "Edge-to-center(mm)", "Edge-to-center (mm)", "edge to center (mm)",
    "vzdialenosť od začiatku v mm", "vzdialenost od zaciatku v mm",
    "distance (mm)", "vzdialenosť (mm)", "vzdialenost (mm)", "x [mm]", "x_mm"
]
HV_CANDS = [
    "HV0.1", "HV 0.1", "HV0,1", "HV 0,1",
    "Vickers HV0.1", "hardness HV0.1", "tvrdosť HV0.1", "tvrdomer HV0.1", "HV"
]

PREFERRED_SAMPLES = ["AS", "SR400_1h", "SR450_1h", "SR500_1h", "SR550_1h", "SA1100_15min"]

# === 3) Read workbook → long dataframe (sample, x, HV) ===
xls = pd.ExcelFile(filename)
rows = []
for sheet in xls.sheet_names:
    df_raw = pd.read_excel(xls, sheet_name=sheet).dropna(axis=1, how="all")
    if df_raw.empty or len(df_raw.columns) < 2:
        continue
    try:
        dist_col = resolve_col(df_raw.columns, DIST_CANDS)
        hv_col   = resolve_col(df_raw.columns, HV_CANDS)
    except KeyError:
        print(f"⚠️  Skipping “{sheet}” (required columns not found). Columns: {list(df_raw.columns)}")
        continue

    d = df_raw[[dist_col, hv_col]].copy()
    d[dist_col] = pd.to_numeric(d[dist_col].astype(str).str.replace(",", "."), errors="coerce")
    d[hv_col]   = pd.to_numeric(d[hv_col].astype(str).str.replace(",", "."), errors="coerce")
    d = d.dropna(subset=[dist_col, hv_col])
    if d.empty:
        continue

    d.columns = ["Edge_to_center_mm", "HV01"]
    d["sample"] = sheet
    rows.append(d)

if not rows:
    raise RuntimeError("No usable sheets/columns found. Expect 'Edge-to-center(mm)' and 'HV0.1'.")

df = pd.concat(rows, ignore_index=True)

# === 4) Deduplicate: keep FIRST per (sample, x rounded to 0.01 mm) ===
df2 = (df.assign(xr=df["Edge_to_center_mm"].round(2))
         .drop_duplicates(subset=["sample", "xr"])
         .rename(columns={"xr": "x_plot"}))

# Order legend: preferred first, then any remaining sheets from workbook order
present   = [s for s in PREFERRED_SAMPLES if s in df2["sample"].unique().tolist()]
remaining = [s for s in xls.sheet_names if s not in present and s in df2["sample"].unique().tolist()]
samples   = present + remaining
df2["sample"] = pd.Categorical(df2["sample"], categories=samples, ordered=True)

# === 5) Styling ===
plt.rcParams.update({
    'font.family': 'serif',
    'font.size': 12,
    'axes.linewidth': AXIS_LINE,
    'mathtext.default': 'regular',
    'lines.linewidth': 7,
    'lines.solid_capstyle': 'round'
})
plt.rcParams['font.serif'] = ["Palatino Linotype", "Palatino", "URW Palladio L"] + plt.rcParams.get('font.serif', [])

# Distinct, colorblind-friendly palette per sample
palette = {
    "AS":          "#1f77b4",  # blue
    "SR400_1h":    "#ff7f0e",  # orange
    "SR450_1h":    "#2ca02c",  # green
    "SR500_1h":    "#d62728",  # red
    "SR550_1h":    "#9467bd",  # purple
    "SA1100_15min":"#8c564b",  # brown
}
default_cycle = ["#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]

# === 6) Plot (deduplicated points) ===
fig = plt.figure(figsize=(10.5, 7.8), dpi=1200)
ax = plt.gca()

for i, s in enumerate(samples):
    sub = df2[df2['sample'] == s].dropna(subset=['x_plot', 'HV01'])
    if sub.empty:
        continue
    color = palette.get(s, default_cycle[i % len(default_cycle)])
    plt.scatter(
        sub['x_plot'], sub['HV01'],
        label=s, color=color, s=MARKER_SIZE, alpha=0.9, edgecolors='none',
        zorder=3  # points above grid
    )

# --- Figure label at the very top center ---
fig.text(0.5, 0.985, FIG_LABEL_TEXT,
         transform=fig.transFigure, ha='center', va='top',
         fontsize=FIG_LABEL_SIZE, fontweight='bold', color=FIG_LABEL_COLOR)

plt.xlabel('Edge-to-center (mm)', fontsize=40, fontweight='bold')
plt.ylabel('HV0.1', fontsize=40, fontweight='bold')

# Spines
for side in ('left', 'bottom', 'right', 'top'):
    ax.spines[side].set_linewidth(AXIS_LINE)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Fixed x; auto y to avoid clipping
ax.set_xticks([0, 1, 2, 3, 4, 5])
ax.set_xlim(0, 5)
ymax = float(df2["HV01"].max())
top  = int(50 * np.ceil(1.05 * ymax / 50.0))
top  = max(top, 600)
ax.set_ylim(0, top)
ax.set_yticks(np.arange(0, top + 1e-9, 200))

# Grid under points
ax.minorticks_on()
ax.grid(True, which='major', axis='both', linestyle=':', linewidth=1.0, alpha=0.55, zorder=0)
ax.grid(True, which='minor', axis='both', linestyle=':', linewidth=0.6, alpha=0.35, zorder=0)

# Ticks bold
ax.tick_params(axis='both', which='major',
               labelsize=38, width=AXIS_LINE, length=10, direction='out', pad=8)
ax.yaxis.set_major_formatter(FormatStrFormatter('%d'))
for ticklbl in (ax.get_xticklabels() + ax.get_yticklabels()):
    ticklbl.set_fontweight('bold')

# Legend centered above x-axis; bigger markers in legend too
leg = plt.legend(loc='upper center', bbox_to_anchor=(0.5, 0.40),
                 ncol=2, fontsize=30, framealpha=0.85, borderpad=0.2, # Decreased borderpad
                 labelspacing=0.5, fancybox=True,
                 markerscale=LEGEND_MARKERSCALE, scatterpoints=1, handletextpad=0.1,
                 columnspacing=0.5)
for txt in leg.get_texts():
    txt.set_fontweight('bold')

plt.tight_layout()
# extra headroom so the top label never clips
plt.subplots_adjust(left=0.14, bottom=0.12, top=0.93)

# === 7) Save THEN show ===
outname = 'Edge_to_center_vs_HV01_scatter_dedup_with_grid_labeled.png'
plt.savefig(outname, format='png', dpi=1200, bbox_inches='tight')
plt.show()

print("🎉 ULTRA-HIGH-DPI PNG SAVED!")
print(f"📄 Filename: {outname}")
print(f"🧭 Y-axis auto top set to: {top} HV")

📁 UPLOAD your workbook (e.g., Hardness_SDSS_3% Ni.xlsx) NOW:


Saving Hardness_SDSS_3% Ni.xlsx to Hardness_SDSS_3% Ni (49).xlsx
📦 Using file on disk: Hardness_SDSS_3% Ni (49).xlsx


🎉 ULTRA-HIGH-DPI PNG SAVED!
📄 Filename: Edge_to_center_vs_HV01_scatter_dedup_with_grid_labeled.png
🧭 Y-axis auto top set to: 600 HV
